In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [12]:
df = pd.read_csv("/content/sales_data.csv")
df.head()

,Date,Sales
0,2013-01-01,48.40
1,2013-01-02,49.63
2,2013-01-03,44.66
3,2013-01-04,56.70
4,2013-01-05,62.87


In [13]:
time = pd.to_datetime(df['Date'])
sales = df['Sales'].values

In [14]:
split_time = int(len(df)*0.8)
train_time = time[:split_time]
val_time = time[split_time:]

x_train = sales[:split_time]
x_val = sales[split_time:]

In [18]:
# Create Windowed dataset
def window_dataset(series,window_size,shuffle_buffer,batch_size):
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + 1, shift = 1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
  dataset = dataset.shuffle(shuffle_buffer).map(lambda window:(window[:-1], window[-1]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset

In [19]:
dataset = window_dataset(x_train,window_size=3,shuffle_buffer=10,batch_size=2)
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>

In [21]:
# build model
window_size = 3
batch_size = 2
shuffle_buffer = 10
model = Sequential([
    Dense(10, input_shape=[window_size], activation='relu'),
    Dense(10, activation='relu'),
    Dense(1)
])
model.compile(loss='mse', optimizer='adam')
history = model.fit(dataset, epochs=100, verbose=0)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


In [22]:
input_batches = [sales[time:time + window_size][np.newaxis] for time in range(len(sales) - window_size)]
inputs = np.concatenate(input_batches, axis=0)
forecast = model.predict(inputs)
results = forecast[split_time-window_size:, 0]

115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [24]:
print(tf.keras.metrics.mse(x_val,results).numpy())
print(tf.keras.metrics.mae(x_val,results).numpy())

38.474
4.939696
